# Data Creation, Exploration and Filtering

In [ ]:
%load_ext autoreload
%autoreload 2
import seaborn as sns
import math
import pandas as pd
import ast
import numpy as np
import matplotlib.pyplot as plt
import json
from helpers import *
data_folder = './DATA/'

# Historical Timeline
In our analysis we want to see how representation of women in american movies might be impacted by historical events. We therefore need to retrieve a history timeline. We decided to use the timeline on https://www.history.com/topics/womens-history/womens-history-us-timeline since it contains major events related to women's history in the US.

In [ ]:
get_history_timeline()

## Importing raw data

In [ ]:
movies_metadata = pd.read_csv(data_folder+'movie.metadata.tsv', header=None, sep='\t', names=['wikipedia_ID','freebase_ID','name','release_date', 'box_office_revenue', 'runtime', 'languages', 'countries', 'genre'], parse_dates=False)
movies_metadata.head(5)

In [ ]:
character_metadata = pd.read_csv(data_folder+'character.metadata.tsv', header=None, sep='\t', names=['wikipedia_ID','freebase_ID','release_date', 'character_name', 'actor_birthday', 'actor_gender', 'actor_height', 'actor_ethnicity', 'actor_name' , 'actor_age','freebase_character_actor_mapID', 'freebase_character_ID', 'freebase_actor_ID' ], parse_dates=False)
character_metadata.head(5)

In [ ]:
plot_summaries = pd.read_csv(data_folder+'plot_summaries.txt',sep='\t', header=None, names=['wikipedia_ID', 'plot_summary'] )
plot_summaries.head()

## Movie dataset: Filtering

**How many movies do we have?**

In [ ]:
number_of_data= movies_metadata.count()
number_of_data

**Where is data mostly missing ?**

In [ ]:
proportion_of_each_data = number_of_data/number_of_data['wikipedia_ID']*100
proportion_of_each_data

We see that the box office column is a column with a lot of missing data, it is only present for 10% of movies. This will not impact our analysis as we are not planning on using the box office as a metric.
One caveat of these results is that an empty list will not be considered as a missing. So for languages, countries and genres this percentages do not really represent the missing data.

Given we will perform our analysis on American movies we will only keep the data for movies from the US

In [ ]:
us_movies_metadata = movies_metadata[movies_metadata['countries'].str.contains('United States of America', case=False)].copy()
us_movies_metadata

How many movies did we keep?

In [ ]:
print(us_movies_metadata.count())

In [ ]:
print('Total Number of movies', number_of_data.wikipedia_ID)
print('Number of non-US movies', number_of_data.wikipedia_ID - us_movies_metadata.wikipedia_ID.count())

In [ ]:
#Let's remove the dictionaries from the countries, the genre and languages of our data
us_movies_metadata['countries'] = us_movies_metadata['countries'].apply(lambda x: list(ast.literal_eval(x).values()))
us_movies_metadata['genre'] = us_movies_metadata['genre'].apply(lambda x: list(ast.literal_eval(x).values()))
us_movies_metadata['languages'] = us_movies_metadata['languages'].apply(lambda x: list(ast.literal_eval(x).values()))

# convert dates to same format and remove movies with unknown movie release dates
us_movies_metadata['release_date'] = us_movies_metadata['release_date'].astype(str)
us_movies_metadata['release_date'] = pd.to_datetime(us_movies_metadata['release_date'],format='mixed', errors='coerce')
us_movies_metadata = us_movies_metadata.dropna(subset=['release_date']).copy()

Let's see what languages we still have in our data

In [ ]:
us_movies_metadata["languages"].explode().unique()

Of interest for us are: English Language, Silent Film, American English, Old English Language (for example Indian English is not very relevant for our analysis, as well as native american languages like Navajo or Hawaiian language). So let us remove any movie that doesn't have English Language, Silent Film, American English, Old English Language, or American Sign Language as one of their language

In [ ]:
us_english_movies_metadata = us_movies_metadata[movies_metadata['languages'].str.contains('English Language' or 'Silent Film' or 'American English' or 'Old English Language' or 'American Sign Language', case=False)].copy()

Did we remove any movies by doing that?

In [ ]:
print('We had',us_movies_metadata.wikipedia_ID.count(), 'movies in our U.S data frame')
print('We now have', us_english_movies_metadata.wikipedia_ID.count(), 'in our English US data frame')
print('So there are',us_movies_metadata.wikipedia_ID.count()-us_english_movies_metadata.wikipedia_ID.count(), 'movies that were from the US but not in any of the languages we decided to consider.')
print('We also see that in the movies we removed, there are', np.sum(us_movies_metadata['languages'].apply(lambda x: len(x)==0)), 'movies that lack languages. For the next milestone we will thus find out the languages of these movies to complete the dataset.')

**Now let us have a look at the genres we have in our dataset**

In [ ]:
us_english_movies_metadata["genre"].explode().unique()

As we can see we have a massive amount (342) of genres, so at some point we will need to filter or chose certain genres to analyze.


**Plot of the number of movies per year**

In [ ]:
#us_english_movies_metadata['release_date'] = pd.to_datetime(us_english_movies_metadata['release_date'],format='mixed', errors='coerce')
#us_english_movies_metadata

In [ ]:
# Group movies by release year
movies_grouped_by_year = us_english_movies_metadata.groupby(us_english_movies_metadata['release_date'].dt.year)

#number of movies per year
number_movies_per_year = movies_grouped_by_year.size()

plt.figure(figsize=(10, 6))
number_movies_per_year.plot(kind='line', color = 'cornflowerblue')
plt.title('Number of Movies Released per Year')
plt.xlabel('Year')
plt.ylabel('Number of Movies')
plt.grid(True)
plt.show()

We can see on this plot that before 1912 we have very little movies. We will thus only keep movies from 1912 and later years.

In [ ]:
us_english_movies_metadata = us_english_movies_metadata[us_english_movies_metadata['release_date'].dt.year >= 1912]

Let us merge this movies dataframe with the synopses

In [ ]:
us_english_movies_synopsis = pd.merge(us_english_movies_metadata, plot_summaries, how='left', on = 'wikipedia_ID')

In [ ]:
us_english_movies_synopsis

Do we have a lot of missing summaries?

In [ ]:
missing_plot = us_english_movies_synopsis['plot_summary'].isna().sum()
number_of_movies = len(us_english_movies_synopsis['plot_summary'])
print('We only have', number_of_movies- missing_plot, 'movies out of',number_of_movies, "with a summary, which means we only have 60.59% of the movies with the plotline. However, the plot summary analysis might be too costly to run on a lot of summaries. It might therefore be reasonable not to complete the missing plot summaries." )

# Character Dataset: Filtering

Let's start by checking the completeness of the character dataset:

In [ ]:
number_of_data_character= character_metadata.count()
proportion_of_each_data_character = number_of_data_character/number_of_data_character['wikipedia_ID']*100
proportion_of_each_data_character

The data seems pretty complete regarding the release date of the movies, actor's gender and actor's age. Nevertheless, we need  to complete the dataset with more character's name to be able to analyze deeper the personnas associated with them and use this result in our analysis. getting also more data on actors' ethnicity would also be of choice as this will allow us to extend our analysis to other minorities such as people of color.

In [ ]:
# remove unknown release dates
character_metadata = character_metadata.dropna(subset='release_date')
# convert the release_date to a date format (YYYY/MM/DD but any date that initially only has a year will be converted to YYYY/01/01)
character_metadata['release_date'] = character_metadata['release_date'].astype(str)
#character_metadata['actor_birthday'] = character_metadata['actor_birthday'].astype(str)
character_metadata['release_date'] = pd.to_datetime(character_metadata['release_date'], format = 'mixed', errors='coerce').dt.year
#character_metadata['actor_birthday'] = pd.to_datetime(character_metadata['actor_birthday'], format = 'mixed', errors='coerce').dt.year

First we will only keep the characters related to the movies we previously filtered

In [ ]:
us_movie_mask = character_metadata['wikipedia_ID'].isin(us_english_movies_metadata['wikipedia_ID'])
character_metadata = character_metadata[us_movie_mask]

Let's filter the character metadata to remove actors with a gender differing from F or M. However, let's first explore the values that "gender" take.

In [ ]:
# Check values in the "actor_gender" column
character_metadata.loc[character_metadata.actor_gender.isna(), "actor_gender"] = "NA"
display(character_metadata.groupby("actor_gender").count().actor_name)

As expected, actor's gender is either F (female), M (male) or NA (not mentionned). Nevertheless, this check was necessary in order to make sure no one identified as non-binary for example. Let's now remove the actors with NA as gender as well as all the characters in movies that have no release date as our analysis is related to time.


In [ ]:
# remove any value that is not "F" or "M" in the "gender" column or the character metadata, keeps the rest
character_metadata_noNA_gender = character_metadata[character_metadata["actor_gender"].isin(["F", "M"])].copy()
# drop the characters in movies with no release date
character_metadata_noNA_genderYear = character_metadata_noNA_gender.dropna(subset=['release_date']).copy()

display(character_metadata_noNA_genderYear)
print('From the original character dataset, we dropped ', character_metadata.shape[0]-character_metadata_noNA_genderYear.shape[0], 'characters either because the movie had no release date or the gender of the actor was not specified, out of ', character_metadata.shape[0], 'characters initially.')

### Merging personas with character dataset

In [ ]:
# Read the data from the text file, comma is the delimiter
personnas_metadata = pd.read_csv(data_folder + 'tvtropes.clusters.txt', names=['personnas', 'data'], header=None, sep='\t')

# Replace single quotes with a placeholder character, to avoid error in parsing
personnas_metadata['data'] = personnas_metadata['data'].str.replace("'", "@@QUOTE@@")
personnas_metadata['data'] = personnas_metadata['data'].apply(lambda x: x.replace("@@QUOTE@@", "'"))

# Function to load JSON strings
def json_loads(x):
    try:
        return json.loads(x)
    except (ValueError, SyntaxError):
        print(f"Error decoding JSON: {x}")
        return None

# Parse the JSON-like strings and split them into separate columns
personnas_metadata['data'] = personnas_metadata['data'].apply(json_loads)
personnas_metadata = pd.concat([personnas_metadata.drop('data', axis=1), personnas_metadata['data'].apply(pd.Series)], axis=1)

#Rename the id column so it matches  with the character dataset to be able to merge (information from the README file) --> maps the actor with its character in the character metadata dataset
personnas_metadata.rename(columns={'id': 'freebase_character_actor_mapID'}, inplace=True)

display(personnas_metadata)

Let's merge this data with the character data we have on freebase_character_actor_mapID as this relates the personnas with the actor and the character they play in a specific movie.

In [ ]:
# only add the column personnas to the characters dataset
character_metadata_noNA_genderYear_personnas = character_metadata.merge(personnas_metadata[['freebase_character_actor_mapID', 'personnas']],on=["freebase_character_actor_mapID"],how="left").copy()

display(character_metadata_noNA_genderYear_personnas)

Finally, we will add the name of the movies to the dataset to facilitate merges with datasets that do not have wikipedia ID.

In [ ]:
character_metadata_noNA_genderYear_personnas = pd.merge(character_metadata_noNA_genderYear_personnas, movies_metadata[['wikipedia_ID', 'name']], how='left', on='wikipedia_ID')

# Complementing the dataset after 2010
The provided dataset does not contain information on recent movies. We thus decided to complement it using IMDB data to be able to also perform our analysis in recent years. We have two main datasets to complete: the movie dataset and the character dataset. To do so, we used the data available on https://datasets.imdbws.com/ and the library Cinemagoer that can retrieve information on IMDB.

### A) Complementing movie data
To complete the movie dataset we used the following folders need to be downloaded from https://datasets.imdbws.com/, unzipped and placed in the /DATA folder:
- title.basics.tsv.gz
- title.akas.tsv.gz

However, these files are missing a lot information that we need for our analysis (plot summaries, countries, languages). We will therefore also use the library Cinemagoer to retrieve these information. In order to not have too many useless requests to IMDB through Cinemagoer, we use the datasets title.basics and titles.akas to get a list of ID of movies we are interested in.

In [ ]:
titles_dataset = pd.read_csv(data_folder + 'title.basics.tsv/data.tsv', sep='\t')
movie_IDs = filter_titles_IDs(titles_dataset)

In [ ]:
print(len(movie_IDs)) # we still have 204'389

Taking all of these movies from IMDB would take too much time. These IDs contain movies from a lot of different countries and we are only interested in american movies. We do not have the 'country' information in the downloaded datasets but we do have the 'original title' and the 'american title' in the title.akas dataset. We will use this dataset to find movies in which the original title is the same as the american one. Thus we can already remove some movies that we know are probably not american. We will of course keep a lot of non-american movies, but we can filter those out later.


In [ ]:
titles_akas_dataset = pd.read_csv(data_folder + 'title.akas.tsv/data.tsv', sep='\t')
original_titles = titles_akas_dataset[titles_akas_dataset['isOriginalTitle']==1]['title']
# get all only the lines where the title is the same as the original title
titles_akas_dataset_filtered = titles_akas_dataset[titles_akas_dataset['title'].isin(original_titles)]
# get only the movies where the US title is the same as the original title
titles_akas_dataset_filtered = titles_akas_dataset_filtered[titles_akas_dataset_filtered['region'] == 'US']

In [ ]:
# get only IDs that are in both datasets
common_ids = titles_akas_dataset_filtered[titles_akas_dataset_filtered['titleId'].isin(movie_IDs)]
common_ids = common_ids['titleId'].drop_duplicates()
print(len(common_ids)) # 39'595 movies left

We will now retrieve information of these 39'595 movies directly from IMDB using Cinemagoer

!! THE FOLLOWING CELL TAKES A LONG TIME TO RUN !!
Since it has already been run once and the data was saved, there is no need to run it anymore and it is thus commented.

In [ ]:
#get_IMDB_movies_data(common_ids)

In [ ]:
IMDB_movie_data = pd.read_csv(data_folder + 'IMDB_movies_2010-2022.csv')
IMDB_movie_data_filtered = filter_IMDB_movie_dataset(IMDB_movie_data)

In [ ]:
IMDB_movie_data_filtered

We now have a datasets containing all the needed information on movies from 2010-2022. We just need to merge it with the provided dataset.

In [ ]:
provided_data = us_english_movies_synopsis
# change the release date to just the year to be able to merge more easily and convert it to the same format of the other df
provided_data['release_date'] = provided_data['release_date'].dt.year
# country and languages are not needed anymore
provided_data = provided_data.drop(columns=['languages', 'countries'])
IMDB_movie_data_filtered = IMDB_movie_data.drop(columns=['languages', 'countries'])

In [1]:
# merge IMDB and provided dataset
movie_data = pd.merge(provided_data, IMDB_movie_data_filtered, on=['name', 'release_date'], how='outer').copy()
duplicated_cols = ['plot_summary', 'genre']
movie_data = remove_duplicated_columns(movie_data, duplicated_cols)
# save data
#movie_data.to_csv(data_folder + 'movie_data.csv')

NameError: name 'pd' is not defined

In [ ]:
movie_data

We now have a data sets containing movies until 2022 that is ready for our analysis!


### B) Complementing characters' data
To complete the character dataset the following folders need to be downloaded from https://datasets.imdbws.com/, unzipped and placed in the /DATA folder:
- title.principals.tsv.gz
- name.basics.tsv.gz

In [ ]:
# load characters info
characters_data = pd.read_csv(data_folder + 'title.principals.tsv/data.tsv', sep='\t')
IMDB_ids = IMDB_movie_data_filtered['IMDB_ID']
# only keep characters of the filtered movies
characters_data_filtered = clean_IMDB_character_dataset(characters_data, IMDB_ids)

In [ ]:
# load actors info
actors_data = pd.read_csv(data_folder + 'name.basics.tsv/data.tsv', sep='\t')
# remove useless columns
actors_data = actors_data.drop(columns=['deathYear', 'primaryProfession', 'knownForTitles'])

We now need to merge all the information we have on the characters and the actors. There are some information we still need to add to the dataframe: release date, actor age, movie name. These will be added by merging with the movie dataset created above.

In [ ]:
# merge actor data and movie data on character data
IMDB_characters_data = merge_datasets_characters(characters_data_filtered, actors_data, IMDB_movie_data_filtered)
IMDB_characters_data.loc[IMDB_characters_data['character_name'] == '\\N', 'character_name'] = None
#IMDB_characters_data['release_date'] = IMDB_characters_data['release_date'].astype(float) #so that it's the same type as the provided data

In [ ]:
IMDB_characters_data

We can now merge this IMDB character dataset with the provided character dataset

In [ ]:
provided_characters = character_metadata_noNA_genderYear_personnas
#change birthday into birth year
#provided_characters['actor_birthday'] = [x.year if pd.notna(x) else x for x in provided_characters['actor_birthday']]
#provided_characters['release_date'] = provided_characters['release_date'].dt.year

In [ ]:
#first put everything in lower case
provided_characters['character_name'] = name_to_lowercase(provided_characters, 'character_name')
provided_characters['actor_name'] = name_to_lowercase(provided_characters, 'actor_name')
IMDB_characters_data['character_name'] = name_to_lowercase(IMDB_characters_data, 'character_name')
IMDB_characters_data['actor_name'] = name_to_lowercase(IMDB_characters_data, 'actor_name')
#provided_characters['name'] = name_to_lowercase(provided_characters, 'name')
#IMDB_characters_data['name'] = name_to_lowercase(IMDB_characters_data, 'name')
# merge datasets
characters_data = pd.merge(provided_characters, IMDB_characters_data, on=['name', 'character_name', 'actor_name', 'release_date'], how='outer').copy()
# remove columns that were duplicated
duplicated_cols = ['actor_gender', 'actor_age']
characters_data= remove_duplicated_columns(characters_data, duplicated_cols)
# save file
#characters_data.to_csv(data_folder + 'characters_data.csv', index=False)

In [ ]:
characters_data

## First Analysis of characters

**What's the proportion of female actors in movies per year?**
We wonder if the proportion of female actors' increase over the years, which is what we would expect.

In [ ]:
# group the movies by year
#characters_data['release_date'] = pd.to_datetime(characters_data['release_date'])
characters_grouped_by_year = characters_data.groupby(characters_data['release_date'])

# Calculate the proportion of men and women per year
proportion_female = characters_grouped_by_year['actor_gender'].apply(lambda x: (x == 'F').sum() / len(x))
proportion_male = 1 - proportion_female

# Plotting the proportion of men vs women per year as a stacked bar chart
plt.figure(figsize=(12, 6))
plt.bar(proportion_male.index, proportion_male.values, bottom=proportion_female.values, label='Male actors', color='skyblue')
plt.bar(proportion_female.index, proportion_female.values, label='Female actors', color='pink')
plt.title('Proportion of Men vs Women in Movies per Year', fontsize=14)
plt.xlabel('Year', fontsize=12)
plt.ylabel('Proportion of total actors', fontsize=12)
plt.legend()
plt.show()

Interestingly, the proportion of women over the decades stayed fairly constant, close to 40%. This constrasts with what we expected as we would think that women representation would get better over the years.  **Questions for the group: should we remove data from before 1910 as so low that doesn't make sense?**

Nevertheless, this result doesn't mean that each movie has approximately 40% of women in their cast. This result might be biased by movies that have an extremely low/high proportion of women versus movies that have a balanced actors cast. To determine if that is the case, let's plot the proportion of movies per year that have 0-25% women, 25-50%, 50-75% or 75-100% women. Are female actors represented equally in every movies or is there a discrepency between movies released in the same year?

In [ ]:
# number of movies per year
number_movies_per_year_character = characters_grouped_by_year.size()

# group by year and by movie
characters_grouped_by_year_and_movie = characters_data.groupby([characters_data['release_date'], 'name'])

# Calculate the proportion of men and women per year per movie
proportion_female_per_movie = characters_grouped_by_year_and_movie['actor_gender'].apply(lambda x: (x == 'F').sum() / len(x))
proportion_male_per_movie = 1 - proportion_female_per_movie

# Convert the Series to a DataFrame for easier data handling in later analysis
female_proportions = proportion_female_per_movie.reset_index()

# Rename the columns if needed
female_proportions.columns = ['release_date', 'name', 'proportion_female']

In [ ]:
#Lists to store in which category each movie belongs to per year
below_25_per_year = []
quarter_50_per_year = []
half_75_per_year = []
above_75_per_year = []

# Iterate through each year in the index of female_proportions
for year, proportions in female_proportions.groupby('release_date'):

    # Count movies in different categories for the current year
    below_25 = len(proportions[proportions['proportion_female'] <= 0.25])
    quarter_50 = len(proportions[(proportions['proportion_female'] > 0.25) & (proportions['proportion_female'] <= 0.5)])
    half_75 = len(proportions[(proportions['proportion_female'] > 0.5) & (proportions['proportion_female'] <= 0.75)])
    above_75 = len(proportions[proportions['proportion_female'] > 0.75])

    # Calculate proportions for each category
    total_movies = len(proportions)
    proportion_below_25 = below_25 / total_movies
    proportion_quarter_50 = quarter_50 / total_movies
    proportion_half_75 = half_75 / total_movies
    proportion_above_75 = above_75 / total_movies

    # Append proportions to respective lists
    below_25_per_year.append(proportion_below_25)
    quarter_50_per_year.append(proportion_quarter_50)
    half_75_per_year.append(proportion_half_75)
    above_75_per_year.append(proportion_above_75)


In [ ]:
# Extract unique years from the 'release_date' column of the female_proportions DataFrame
unique_years = female_proportions['release_date'].unique()

# Convert the unique years to a list
years = list(unique_years)

# Data for stack plot
categories = ['Below 25%', '25-50%', '50-75%', 'Above 75%']
data = [below_25_per_year, quarter_50_per_year, half_75_per_year, above_75_per_year]

# Create a stack plot
plt.figure(figsize=(10, 6))
plt.stackplot(years, data, labels=categories, alpha=0.8)
plt.xlabel('Year')
plt.ylabel('Proportion of movies in a given year')
plt.title('Proportion of Movies with Different Percentages of Female Actors')
plt.legend(loc='upper left')
plt.show()

We can observe that a vast majority of movies have less than 50% of women in their cast. Actually, over the years, a vast majority of movies (approximately 80%) have less than 50% female actors in their cast. Thus, even though overall the proportion of women seems to be close to 40% over the years, their are largely underrepresented in the vast majority of movies.

**What's the age difference of female actors in movies compared to male actors over the years? Are female actors significantly younger than male actors?**

In [ ]:
# drop the actors with NaN as age but also actors with negative ages (which appeared in 1910)
character_age = characters_data.dropna(subset=['actor_age']).copy()
character_age = character_age[character_age['actor_age'] >= 0] # remove actors with a negative age

In [ ]:
# Group by year and gender
characters_age_by_year_gender = character_age.groupby([character_age['release_date'], 'actor_gender'])

# Calculate average age and standard deviation of age for each gender per year
gender_age_stats_per_year = characters_age_by_year_gender['actor_age'].agg(['mean', 'sem'])

# Reset the index to make the grouped columns accessible for further analysis or visualization
gender_age_stats_per_year.reset_index(inplace=True)


In [ ]:
# Filter data for men and women separately for easier handling
men_data = gender_age_stats_per_year[gender_age_stats_per_year['actor_gender'] == 'M'].copy()
women_data = gender_age_stats_per_year[gender_age_stats_per_year['actor_gender'] == 'F'].copy()

# Calculate 95% confidence interval for men
men_data['lower_bound'] = men_data['mean'] - 1.96 * men_data['sem']
men_data['upper_bound'] = men_data['mean'] + 1.96 * men_data['sem']

# Calculate 95% confidence interval for women
women_data['lower_bound'] = women_data['mean'] - 1.96 * women_data['sem']
women_data['upper_bound'] = women_data['mean'] + 1.96 * women_data['sem']

# Plotting for men
plt.figure(figsize=(10, 6))
plt.plot(men_data['release_date'], men_data['mean'], label='Men Mean Age', color='skyblue')
plt.fill_between(men_data['release_date'], men_data['lower_bound'], men_data['upper_bound'], alpha=0.3, color='skyblue')

# Plotting for women
plt.plot(women_data['release_date'], women_data['mean'], label='Women Mean Age', color='pink')
plt.fill_between(women_data['release_date'], women_data['lower_bound'], women_data['upper_bound'], alpha=0.3, color='pink')
plt.xlabel('Year')
plt.ylabel('Average age of actors')
plt.title('Average age of actors per year per gender with 95% Confidence Interval')
plt.legend()
plt.grid(True)
plt.show()

Throughout the years, female actors are significantly younger than men at the 5% significance level.


**TO DO: MERGE WITH THE PERSONNAS TO ANALYSE**
regarder les roles opposés (dumb blonde vs genius) et voir si plus représenté dans un genre + enlever les genre ou ça fait pas sens de comparer genre (e.g. films de guerre, documentaires... car ce sont des évènements historique donc on peut rien y faire))

## Personas Analysis

**What personnas are associated with movie characters interpreted by male and female actors? Are female characters (so female actors) mostly associated woth beauty, stupidity, weakness and sexualized and men mostly associated with heroism, adventure, strenght...?**

In [ ]:
# For the following analysis, drop all the characters that don't have personnas
characters_data_personnas_noNA = characters_data.dropna(subset=['personnas']).copy()
display(characters_data_personnas_noNA)

From the size of the datasets, we can see that one personnas has been dropped because there were no matches between the freebase_character_actor_mapID in the characters data.


In [ ]:
# Check values in the "actor_gender" column to make sure it is populated
characters_data_personnas_noNA.loc[characters_data_personnas_noNA.actor_gender.isna(), "actor_gender"] = "NA"
display(characters_data_personnas_noNA.groupby("actor_gender").count().actor_name)

From this very simple line, we can already see that out of the 500 characters, only 69 are played by women, which is already a poor representation.

In [ ]:
# split the dataframe in 2: one for males and one for women, which is easier for the subsequent analysis
male_personnas_data = characters_data_personnas_noNA[characters_data_personnas_noNA['actor_gender'] == 'M'].copy()
female_personnas_data = characters_data_personnas_noNA[characters_data_personnas_noNA['actor_gender'] == 'F'].copy()

In [ ]:
# print the female personnas with their number of occurance
unique_female_personnas_counts = female_personnas_data["personnas"].explode().value_counts()
# Sort the unique_personnas_counts Series by the count in descending order
sorted_female_personnas_counts = unique_female_personnas_counts.sort_values(ascending=False)
sorted_female_personnas_counts

In [ ]:
# Do the same for men
unique_male_personnas_counts = male_personnas_data["personnas"].explode().value_counts()
# Sort the unique_personnas_counts Series by the count in descending order
sorted_male_personnas_counts = unique_male_personnas_counts.sort_values(ascending=False)
print(sorted_male_personnas_counts.head(20))

In [ ]:
# Extract the index (unique 'personnas') from both DataFrames
female_personnas = sorted_female_personnas_counts.index
male_personnas = sorted_male_personnas_counts.index

# Find the intersection between the two sets of 'personnas'
common_personnas = set(female_personnas) & set(male_personnas)

# Create a list of with the common personnas and their count in the male and female data
common_personnas_data = [
    {
        'personna': personna,
        'female_count': sorted_female_personnas_counts.get(personna, 0),
        'male_count': sorted_male_personnas_counts.get(personna, 0),
        'association_with_F_compared_to_M' : (sorted_female_personnas_counts.get(personna, 0)/len(female_personnas_data)) / (sorted_male_personnas_counts.get(personna, 0)/len(male_personnas_data))
    }
    for personna in common_personnas
]

# Create a DataFrame from the list of dictionaries
common_personnas_df = pd.DataFrame(common_personnas_data)
display(common_personnas_df)

This dataframe shows which personnas are common for male and female as well as the count of each gender. The last colomn normalizes the counts to be able to compare them between females and males (as there are 69 females vs 431 males). It is sticking how "dumb_blond" is more than 64 times more for associated with women than men. Overall, terms with a bad connotation (broken_bird, dumb_blonde, brainless_beauty or bully) are much more associated to women than men. However, it is important to remember that the analysis is made only on 500 characters.

